In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8syjuqb_/unsloth_864f045a36074c03940c4186a9f86b9f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8syjuqb_/unsloth_864f045a36074c03940c4186a9f86b9f
  Resolved https://github.com/unslothai/unsloth.git to commit e41cc4093c70095e4aef390c8afae85c38aa4eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from unsloth import FastLanguageModel
from unsloth import to_sharegpt, standardize_sharegpt
from unsloth import is_bfloat16_supported
import json
from datasets import Dataset
import pandas as pd
import numpy as np
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
MAX_LENGTH = 2048
DTYPE = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = MAX_LENGTH,
    dtype = DTYPE,
    load_in_4bit = load_in_4bit,
    use_gradient_checkpointing = "unsloth",
)

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
peft_model = FastLanguageModel.get_peft_model(
              model = model,
              r = 16,
              target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                                "gate_proj", "up_proj", "down_proj",],
              lora_alpha = 16,
              lora_dropout = 0,
              bias = "none",
              use_gradient_checkpointing = "unsloth",
              random_state = 8080,
              use_rslora = False,
              loftq_config = None
          )

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Data

In [6]:
DATA_PATH = "/content/drive/MyDrive/FinancialAdvisorProject/dataset_final.json"

In [7]:
f = open(DATA_PATH)
data = f.read()

In [8]:
finance_data = json.loads(data)

In [9]:
finance_data[0]

{'about_me': 'I am a 21 year old college student.I was thinking of investing in the stock market.',
 'context': 'Meta fires 10k employees.\nMeta about to release Threads app.\nZuckerberg to visit China soon',
 'response': 'Monitor Meta due to layoffs and app release. Wait for stability before investing. Recent layoffs and impending app release may impact stock. Wait for stability.',
 'question': 'Is Meta a good stock to buy?'}

In [10]:
merged_prompt = """
     You are a Financial Advisor and your main role is to be give suggestions. \
     User will tell about themselves first and ask you a question whether it is \
     good to invest in the stock. You will be given context as well which is the \
     latest news from Alpaca and based on the news and history, you need to \
     suggesting the user whether it is good to buy a stock or not. Along with \
     that, you will also provide the appropriate reasoning based on the context. \
     Below is the input
     {about_me}{question}{context}
"""

In [11]:
finance_data = Dataset.from_pandas(pd.DataFrame(data=finance_data))

In [12]:
dataset = to_sharegpt(
    dataset = finance_data,
    merged_prompt = merged_prompt,
    output_column_name = "response",
    conversation_extension = 5
)

Merging columns:   0%|          | 0/84 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/84 [00:00<?, ? examples/s]

In [13]:
dataset = standardize_sharegpt(dataset = dataset)

Standardizing format:   0%|          | 0/84 [00:00<?, ? examples/s]

# Training

In [14]:
trainer = SFTTrainer(
    model = peft_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    max_seq_length = MAX_LENGTH,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 25,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/84 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 84 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 25
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.075000
2,1.050800
3,1.068000
4,1.046400
5,0.995500
6,0.936300
7,0.898100
8,0.750100
9,0.712700
10,0.672400


# Inference

In [16]:
prompt_template = """
     You are a Financial Advisor and your main role is to be give suggestions. \
     User will tell about themselves first and ask you a question whether it is \
     good to invest in the stock. You will be given context as well which is the \
     latest news from Alpaca and based on the news and history, you need to \
     suggesting the user whether it is good to buy a stock or not. Along with \
     that, you will also provide the appropriate reasoning based on the context. \
     Below is the input
     # ABOUT ME:
     {}
     # QUESTION:
     {}
     # CONTEXT:
     {}
     # RESPONSE:
     {}"""

In [20]:
prompt_template.format(
        "I am a 42 year old supply chain manager.I'm interested in investing in logistics technology.",
        "What are the innovations in supply chain management?",
        "Logistics technology disrupts traditional supply chain operations with automation, robotics, and predictive analytics.E-commerce growth accelerates demand for last-mile delivery solutions and warehouse robotics.Investors bet on supply chain startups optimizing transportation, inventory management, and order fulfillment processes.", # input
        ""
    )

"\n     You are a Financial Advisor and your main role is to be give suggestions.      User will tell about themselves first and ask you a question whether it is      good to invest in the stock. You will be given context as well which is the      latest news from Alpaca and based on the news and history, you need to      suggesting the user whether it is good to buy a stock or not. Along with      that, you will also provide the appropriate reasoning based on the context.      Below is the input \n     # ABOUT ME:\n     I am a 42 year old supply chain manager.I'm interested in investing in logistics technology.\n     # QUESTION:\n     What are the innovations in supply chain management?\n     # CONTEXT:\n     Logistics technology disrupts traditional supply chain operations with automation, robotics, and predictive analytics.E-commerce growth accelerates demand for last-mile delivery solutions and warehouse robotics.Investors bet on supply chain startups optimizing transportation, inv

In [ ]:
model

In [24]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(peft_model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "I am a 42 year old supply chain manager.I'm planning to invest in CDSL stock",
        "What are your suggestions to invest in CDSL stock for 5 years",
        "CDSL stock is the monopoly market where all the stocks money go through this channel", # input
        ""
    )
], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>
     You are a Financial Advisor and your main role is to be give suggestions.      User will tell about themselves first and ask you a question whether it is      good to invest in the stock. You will be given context as well which is the      latest news from Alpaca and based on the news and history, you need to      suggesting the user whether it is good to buy a stock or not. Along with      that, you will also provide the appropriate reasoning based on the context.      Below is the input 
     # ABOUT ME:
     I am a 42 year old supply chain manager.I'm planning to invest in CDSL stock
     # QUESTION:
     What are your suggestions to invest in CDSL stock for 5 years
     # CONTEXT:
     CDSL stock is the monopoly market where all the stocks money go through this channel
     # RESPONSE:
      CDSL is the primary depository for dematerialized shares in India. It offers secure storage and transfer of securities, ensuring transparency and efficiency in the stock 